In [ ]:
import torch
import torch.optim as optim
from unet_layers import *
import h5py
from torch.utils.data import DataLoader
from torchvision import transforms as T
from tensorboardX import SummaryWriter
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from data_utils import FSdataset,my_RandomCrop,my_normalize
from visualize import show_FS,show_EPI_xu,show_EPI_yv,show_SAI
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"
np.random.seed(100);
torch.manual_seed(100);
torch.backends.cudnn.deterministic = True

In [ ]:
bs_train = 4
bs_val = 10
lr = 1e-3

nF=7
lfsize = [372, 540, 8, 8] #H,W,v,u
#dimensions of Lytro light fields, H,W,nv,nu. 
#Note original Lytro LF has dimension 376X541 X 14 X 14, the paper takes only first 372/540 spatial pixel and central 8 by 8 SAI
#which is being followed here

transform_train = T.Compose([my_normalize(255),my_RandomCrop(256)])
transform_val = T.Compose([my_normalize(255)])

ds_train = FSdataset(lfsize = lfsize, FSdata_path='FS_dataset/FS_dmin_-1_dmax_0.3_nF_7.h5',LFdata_folder='Flowers_8bit/',trainorval='train',transform = transform_train)
ds_val =  FSdataset(lfsize = lfsize, FSdata_path='FS_dataset/FS_dmin_-1_dmax_0.3_nF_7.h5',LFdata_folder='Flowers_8bit/',trainorval='val',transform = transform_val)
writer = SummaryWriter('logs/FS_dmin_-1_dmax_0.3_nF_7/unet_FS2LF_v3/lr_1e-3_bs_train_4_bs_val_10_pin_memory')
#writer = SummaryWriter('logs/test')

train_loader=DataLoader(ds_train, batch_size=bs_train,shuffle=True, num_workers = 1,pin_memory = True)
val_loader=DataLoader(ds_val, batch_size=bs_val,shuffle=False, num_workers = 0,pin_memory = True)

In [ ]:

device = torch.device("cuda")
net = unet_FS2LF_v3(nF=nF,nu=lfsize[3],nv=lfsize[2])
net.to(device)
criterion = nn.L1Loss()
#criterion = nn.MSELoss()
optimizer=optim.Adam(net.parameters(),lr=lr)

In [ ]:
t1 =time.time()
step=0
for epoch in range(50):
    print("Current epoch number%d" %epoch) 
    for idx,data in enumerate(train_loader,0):
        net.train()
        FS,LF=data['FS'].to(device),data['LF'].to(device) # 2019 4 16: check loaded FS, LF dimension and make sure they are compatible with network struture here
        reconLF=net(FS)
        loss=criterion(reconLF,LF)
        print('Loss is %3f' %(loss.item()))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        
        if step % 10 == 0:
            writer.add_scalar('loss', loss.item(), step)    
        if step % 100 == 0:
            print('Train visualization')
            show_SAI(reconLF.detach().cpu(),[(0,0),(5,5)])
            show_EPI_xu(reconLF.detach().cpu(),[(100,0),(200,3)])
        step = step + 1 
        print('step %d, time %f' %(step,time.time()-t1))
    if np.mod(epoch,1)==0: 
        net.eval()
        data=next(iter(val_loader))
        FS,LF=data['FS'].to(device),data['LF'].to(device) 

        with torch.no_grad():
            reconLF=net(FS)
            loss=criterion(reconLF,LF)
        print('Validation visualization')
        show_SAI(reconLF.cpu(),[(0,0),(5,5)])
        show_EPI_xu(reconLF.cpu(),[(100,0),(200,3)])
        print('Val Loss is %3f' %(loss.item()))#for multiple test sample
        
        writer.add_scalar('Val loss', loss.item(), step)
        #Add some visulization here
        
